<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [23]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [24]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [25]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests, time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/124.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://en.wikipedia.org/"
}

s = requests.Session()
s.headers.update(headers)

resp = s.get(static_url, timeout=30)
print("status:", resp.status_code, "final url:", resp.url)

status: 200 final url: https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches
title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


Create a `BeautifulSoup` object from the HTML `response`


In [27]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(resp.text, "html.parser")
print("title:", soup.title.string)

title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [32]:
# Use soup.title attribute
tables = pd.read_html(resp.text, displayed_only=False)

def show_table_overview(tables):
    for i, t in enumerate(tables):
        cols = [str(c) for c in t.columns]
        print(f"{i:>2}: {cols[:8]} ... rows={len(t)}")

def looks_like_launch_table(df):
    cols = " | ".join([str(c) for c in df.columns]).lower()
    must_have = ["flight", "date", "launch", "payload"]  # broad signals
    return all(x in cols for x in must_have)

candidate_idxs = [i for i, t in enumerate(tables) if looks_like_launch_table(t)]
candidate_idxs

idx = candidate_idxs[0]   # if multiple, inspect each with tables[i].head()
launches_raw = tables[idx].copy()

# flatten multiindex columns if any
if isinstance(launches_raw.columns, pd.MultiIndex):
    launches_raw.columns = [" ".join([str(x) for x in c if str(x) != "nan"]).strip()
                            for c in launches_raw.columns]
else:
    launches_raw.columns = [str(c).strip() for c in launches_raw.columns]

# keep the core columns (names vary a bit across revisions)
col_map_candidates = {
    "Flight": ["Flight No.", "Flight", "№", "No."],
    "Date": ["Date and time (UTC)", "Date/time (UTC)", "Date"],
    "LaunchSite": ["Launch site", "Launch site", "Launch Site"],
    "Payload": ["Payload", "Payload(s)"],
    "PayloadMass": ["Payload mass", "Payload mass [kg]", "Mass", "Mass (kg)"],
    "Orbit": ["Orbit"],
    "Customer": ["Customer", "Operator"],
    "Core": ["Core", "Core serial", "Booster"],
    "Landing": ["Landing", "Landing outcome", "Landing outcome"],
}

def pick(df, names):
    for n in names:
        if n in df.columns:
            return n
    return None

cols = {k: pick(launches_raw, v) for k, v in col_map_candidates.items()}
# drop keys that weren't found
cols = {k:v for k,v in cols.items() if v is not None}

launches = launches_raw[list(cols.values())].rename(columns={v:k for k,v in cols.items()})

# basic cleanup: drop wholly empty rows, trim strings
launches = launches.dropna(how="all")
for c in launches.select_dtypes(include="object").columns:
    launches[c] = launches[c].astype(str).str.strip()

launches.head()

,Flight,PayloadMass,Orbit,Customer
0,286,"~16,800 kg (37,000 lb)",LEO,SpaceX
1,286,"Launch of 22 Starlink v2 mini satellites, incl...","Launch of 22 Starlink v2 mini satellites, incl...","Launch of 22 Starlink v2 mini satellites, incl..."
2,287,"1,800 kg (4,000 lb)",GTO,Ovzon
3,287,Broadband internet provider satellite.[26] Fir...,Broadband internet provider satellite.[26] Fir...,Broadband internet provider satellite.[26] Fir...
4,288,"~17,100 kg (37,700 lb)",LEO,SpaceX


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [33]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# find all tables on the page
html_tables = soup.find_all('table')

# Starting from the third table is our target
first_launch_table = html_tables[2]
print(first_launch_table.thead)   # optional: peek at header only

None


Starting from the third table is our target table contains the actual launch records.


In [34]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible sticky-header" id="2025ytd" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_32-1"><a href="#cite_note-booster-32"><span class="cite-bracket">[</span>i<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch<br/>site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_33-1"><a href="#cite_note-Dragon-33"><span class="cite-bracket">[</span>j<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [35]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)   # uses your helper to clean the text
    if name is not None and len(name) > 0:
        column_names.append(name)

Check the extracted column names


In [36]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [44]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [45]:
extracted_row = 0

for table_number, table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    for row in table.find_all("tr"):
        if row.th:  # header cell
            flight_number = row.th.string.strip() if row.th.string else ''
            flag = flight_number.isdigit()
        else:
            flag = False
        
        if flag:
            extracted_row += 1
            row_elems = row.find_all('td')
            
            # --- Flight number
            launch_dict['Flight No.'].append(flight_number)
            
            # --- Date + Time
            datetimelist = date_time(row_elems[0])
            launch_dict['Date'].append(datetimelist[0].strip())
            launch_dict['Time'].append(datetimelist[1].strip() if len(datetimelist) > 1 else "")
            
            # --- Booster version
            bv = booster_version(row_elems[1])
            launch_dict['Version Booster'].append(bv if bv else "")
            
            # --- Launch site
            launch_dict['Launch site'].append(row_elems[2].a.string if row_elems[2].a else "")
            
            # --- Payload
            launch_dict['Payload'].append(row_elems[3].a.string if row_elems[3].a else "")
            
            # --- Payload mass
            launch_dict['Payload mass'].append(get_mass(row_elems[4]))
            
            # --- Orbit
            launch_dict['Orbit'].append(row_elems[5].a.string if row_elems[5].a else "")
            
            # --- Customer
            launch_dict['Customer'].append(row_elems[6].a.string if row_elems[6].a else "")
            
            # --- Launch outcome
            lo = list(row_elems[7].strings)[0].strip()
            launch_dict['Launch outcome'].append(lo)
            
            # --- Booster landing
            bl = landing_status(row_elems[8])
            launch_dict['Booster landing'].append(bl)


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [46]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [47]:
df.to_csv('spacex_web_scraped.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
